# JAMS-Test: evaluation of the jamifier

In [1]:
import os
import sys
import glob

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.append(os.path.join(os.path.dirname(os.getcwd()), "choco"))


# Choose plotting style
# sns.set_style("whitegrid")
# sns.set_style("darkgrid")

In [2]:
col_remapping = {
    "cov": "coverage",
    "acc": "accuracy",
    "err": "error",
    "iden": "identifiers",
    "key_cov": "key coverages",
    "key_err": "key errorr",
    "chord_cov": "chord coverages",
    "chord_err": "chord errors",
}

def preprocess_partition_test_df(partition_test_path:str):

    partition_test_df = pd.read_csv(partition_test_path)
    # Append an extra column with the name of the outer partition
    partition_dir = os.path.dirname(os.path.dirname(partition_test_df["gold"][0]))
    if os.path.basename(partition_dir) == "test":  # still need to go one up
        partition_dir = os.path.dirname(partition_dir)
    partition_name = os.path.basename(partition_dir)
    partition_test_df["partition"] = partition_name

    return partition_test_df


def describe_partitions(partition_res_df:pd.DataFrame, dec=2):

    partition_means = partition_res_df.groupby("partition").mean().round(dec)
    partition_stds = partition_res_df.groupby("partition").std().round(dec)
    # String representation of the combined mean ± std partition-wise
    partition_str = partition_means.astype(str) \
        + " ± " + partition_stds.astype(str)
    partition_str = partition_str.replace("nan ± nan", "-")

    return partition_means, partition_stds, partition_str


def merge_columns(dataframe, remap=col_remapping, sep="_"):

    new_columns = []
    for nested_cname in dataframe.columns:  # split and rename
        outer_cname, inner_cname = tuple(nested_cname.rsplit(sep, 1))
        outer_cname = remap.get(outer_cname, outer_cname)
        inner_cname = remap.get(inner_cname, inner_cname)
        new_columns.append((outer_cname, inner_cname))
    dataframe.columns = pd.MultiIndex.from_tuples(new_columns)


## Processing a single evaluation

In [3]:
test_sample = "../partitions/isophonics/test/test_results_agg.csv"

test_sample_df = pd.read_csv(test_sample)

test_sample_df

,meta_cov,meta_acc,iden_cov,iden_acc,key_cov_time,key_cov_duration,key_cov_value,key_err_time,key_err_duration,key_err_value,chord_cov_time,chord_cov_duration,chord_cov_value,chord_err_time,chord_err_duration,chord_err_value,gold,jamified,type
0,1.0,1.0,NaN,NaN,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,../partitions/isophonics/test/isophonics_164_s...,../partitions/isophonics/choco/jams/isophonics...,audio
1,1.0,1.0,NaN,NaN,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,../partitions/isophonics/test/isophonics_200_s...,../partitions/isophonics/choco/jams/isophonics...,audio
2,1.0,1.0,NaN,NaN,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,../partitions/isophonics/test/isophonics_73_si...,../partitions/isophonics/choco/jams/isophonics...,audio
3,1.0,1.0,NaN,NaN,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,../partitions/isophonics/test/isophonics_163_s...,../partitions/isophonics/choco/jams/isophonics...,audio


## Aggregation of evaluations

In [4]:
test_results = glob.glob("../partitions/*/test/**/test_results_agg.csv", recursive=True)
print(f"Found {len(test_results)} test result CSV files from partitions")

test_results

Found 19 test result CSV files from partitions


['../partitions/weimar/test/test_results_agg.csv',
 '../partitions/ireal-pro/test/playlists/test_results_agg.csv',
 '../partitions/rwc-pop/test/test_results_agg.csv',
 '../partitions/isophonics/test/test_results_agg.csv',
 '../partitions/uspop2002/test/test_results_agg.csv',
 '../partitions/when-in-rome/test/test_results_agg.csv',
 '../partitions/real-book/test/test_results_agg.csv',
 '../partitions/wikifonia/test/test_results_agg.csv',
 '../partitions/jazz-corpus/test/test_results_agg.csv',
 '../partitions/robbie-williams/test/test_results_agg.csv',
 '../partitions/nottingham/test/test_results_agg.csv',
 '../partitions/chordify/test/test_results_agg.csv',
 '../partitions/rock-corpus/test/test_results_agg.csv',
 '../partitions/schubert-winterreise/test/score/test_results_agg.csv',
 '../partitions/schubert-winterreise/test/audio/test_results_agg.csv',
 '../partitions/jaah/test/test_results_agg.csv',
 '../partitions/billboard/test/test_results_agg.csv',
 '../partitions/mozart-piano-sonat

In [5]:
partition_test_dfs = [preprocess_partition_test_df(ptest_path) for ptest_path in test_results]

In [6]:
all_partitions = pd.concat(partition_test_dfs)
# Recompute indexes and drop JAMS-file columns
all_partitions = all_partitions.reset_index(drop=True)
all_partitions = all_partitions.drop(["gold", "jamified"], axis=1)

In [7]:
meta_cols = ["meta_cov", "meta_acc", "iden_cov", "iden_acc", "partition"]
anns_cols = [c for c in all_partitions.columns if c not in meta_cols] + ["partition"]

all_partitions_meta = all_partitions[meta_cols]
all_partitions_anns = all_partitions[anns_cols]

all_partitions_anns_score = all_partitions_anns[all_partitions_anns["type"]=="score"]
all_partitions_anns_audio = all_partitions_anns[all_partitions_anns["type"]=="audio"]

In [8]:
all_partitions

,meta_cov,meta_acc,iden_cov,iden_acc,key_cov_time,key_cov_duration,key_cov_value,key_err_time,key_err_duration,key_err_value,chord_cov_time,chord_cov_duration,chord_cov_value,chord_err_time,chord_err_duration,chord_err_value,type,partition
0,1.000,1.000000,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.000000,audio,weimar
1,1.000,0.951389,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.000000,audio,weimar
2,1.000,1.000000,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.000000,audio,weimar
3,1.000,1.000000,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.000000,audio,weimar
4,1.000,1.000000,NaN,NaN,0.0,1.0,1.0,1.1,0.0,0.0,0.8,1.0,1.0,1.0,0.0,0.000000,score,ireal-pro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,0.875,1.000000,1.0,1.0,1.0,0.0,0.0,0.0,53.0,0.5,1.0,0.5,0.0,0.0,0.4,0.590476,score,mozart-piano-sonatas
68,1.000,1.000000,NaN,NaN,1.0,1.0,1.0,0.0,0.0,0.0,0.8,1.0,1.0,0.2,0.0,0.000000,score,biab-internet-corpus
69,1.000,0.823171,NaN,NaN,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.000000,score,biab-internet-corpus
70,0.800,0.888889,NaN,NaN,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.000000,score,biab-internet-corpus


In [9]:
partitions = list(all_partitions.partition.unique())

print(f"Found {len(partitions)} partitions: {partitions}")

Found 18 partitions: ['weimar', 'ireal-pro', 'rwc-pop', 'isophonics', 'uspop2002', 'when-in-rome', 'real-book', 'wikifonia', 'jazz-corpus', 'robbie-williams', 'nottingham', 'chordify', 'rock-corpus', 'schubert-winterreise', 'jaah', 'billboard', 'mozart-piano-sonatas', 'biab-internet-corpus']


### Check single partition

In [10]:
partition_name = "schubert-winterreise"

all_partitions[all_partitions["partition"]==partition_name]

,meta_cov,meta_acc,iden_cov,iden_acc,key_cov_time,key_cov_duration,key_cov_value,key_err_time,key_err_duration,key_err_value,chord_cov_time,chord_cov_duration,chord_cov_value,chord_err_time,chord_err_duration,chord_err_value,type,partition
52,1.0,1.0,NaN,NaN,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,score,schubert-winterreise
53,1.0,1.0,NaN,NaN,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,score,schubert-winterreise
54,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,audio,schubert-winterreise
55,1.0,1.0,NaN,NaN,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,audio,schubert-winterreise


### Metadata results

In [11]:
_, _, meta_res_str = describe_partitions(all_partitions_meta)
merge_columns(meta_res_str)

In [12]:
meta_res_str

meta              identifiers           
                        coverage     accuracy    coverage   accuracy
partition                                                           
biab-internet-corpus  0.95 ± 0.1  0.92 ± 0.08           -          -
billboard              1.0 ± 0.0    1.0 ± 0.0           -          -
chordify               1.0 ± 0.0    1.0 ± 0.0   1.0 ± 0.0  1.0 ± 0.0
ireal-pro              1.0 ± 0.0    1.0 ± 0.0           -          -
isophonics             1.0 ± 0.0    1.0 ± 0.0           -          -
jaah                  0.8 ± 0.06    1.0 ± 0.0   1.0 ± 0.0  1.0 ± 0.0
jazz-corpus            1.0 ± 0.0    1.0 ± 0.0           -          -
mozart-piano-sonatas  0.88 ± 0.0    1.0 ± 0.0   1.0 ± 0.0  1.0 ± 0.0
nottingham             1.0 ± 0.0    1.0 ± 0.0           -          -
real-book              1.0 ± 0.0    1.0 ± 0.0           -          -
robbie-williams        1.0 ± 0.0    1.0 ± 0.0           -          -
rock-corpus            1.0 ± 0.0    1.0 ± 0.0           -          -
rwc-pop                1.0 ± 0.0    1.0 ± 0.0           -          -
schubert-winterreise   1.0 ± 0.0    1.0 ± 0.0   1.0 ± nan  1.0 ± nan
uspop2002              1.0 ± 0.0  0.97 ± 0.06           -          -
weimar                 1.0 ± 0.0  0.99 ± 0.02   1.0 ± 0.0  1.0 ± 0.0
when-in-rome          0.8 ± 0.06  0.96 ± 0.07           -          -
wikifonia             0.95 ± 0.1   0.95 ± 0.1           -          -

### Annotation results: score

In [13]:
_, _, ann_res_score_str = describe_partitions(all_partitions_anns_score)
merge_columns(ann_res_score_str)

In [14]:
ann_res_score_str

key coverages                             key errorr  \
                              time   duration       value            time   
partition                                                                   
biab-internet-corpus     1.0 ± 0.0  1.0 ± 0.0   1.0 ± 0.0       0.0 ± 0.0   
ireal-pro                0.0 ± 0.0  1.0 ± 0.0   1.0 ± 0.0       1.1 ± 0.0   
jazz-corpus              1.0 ± 0.0  1.0 ± 0.0   1.0 ± 0.0       0.0 ± 0.0   
mozart-piano-sonatas    0.5 ± 0.58  0.0 ± 0.0  0.5 ± 0.58  62.55 ± 125.03   
nottingham               1.0 ± 0.0  1.0 ± 0.0   1.0 ± 0.0       0.0 ± 0.0   
real-book                1.0 ± 0.0  1.0 ± 0.0   1.0 ± 0.0       0.0 ± 0.0   
rock-corpus              1.0 ± 0.0  1.0 ± 0.0   1.0 ± 0.0       0.0 ± 0.0   
schubert-winterreise     1.0 ± 0.0  1.0 ± 0.0   1.0 ± 0.0       0.0 ± 0.0   
when-in-rome             1.0 ± 0.0  1.0 ± 0.0   1.0 ± 0.0       0.0 ± 0.0   
wikifonia                1.0 ± 0.0  1.0 ± 0.0   1.0 ± 0.0       0.0 ± 0.0   

                                                  chord coverages  \
                            duration        value            time   
partition                                                           
biab-internet-corpus       0.0 ± 0.0    0.0 ± 0.0      0.95 ± 0.1   
ireal-pro                  0.0 ± 0.0    0.0 ± 0.0       0.2 ± 0.4   
jazz-corpus                0.0 ± 0.0    0.0 ± 0.0       1.0 ± 0.0   
mozart-piano-sonatas  139.75 ± 83.75  0.25 ± 0.29      0.85 ± 0.3   
nottingham                 0.0 ± 0.0    0.0 ± 0.0     0.75 ± 0.25   
real-book                  0.0 ± 0.0    0.0 ± 0.0       1.0 ± 0.0   
rock-corpus                0.0 ± 0.0    0.0 ± 0.0       1.0 ± 0.0   
schubert-winterreise       0.0 ± 0.0    0.0 ± 0.0       1.0 ± 0.0   
when-in-rome               0.0 ± 0.0    0.0 ± 0.0       1.0 ± 0.0   
wikifonia                  0.0 ± 0.0    0.0 ± 0.0       1.0 ± 0.0   

                                               chord errors               \
                         duration        value         time     duration   
partition                                                                  
biab-internet-corpus    1.0 ± 0.0    1.0 ± 0.0   0.05 ± 0.1    0.0 ± 0.0   
ireal-pro               1.0 ± 0.0    1.0 ± 0.0  1.08 ± 0.05    0.0 ± 0.0   
jazz-corpus             1.0 ± 0.0    1.0 ± 0.0    0.0 ± 0.0    0.0 ± 0.0   
mozart-piano-sonatas  0.88 ± 0.25   0.75 ± 0.5   0.25 ± 0.5  0.15 ± 0.19   
nottingham              1.0 ± 0.0    1.0 ± 0.0   0.85 ± 0.6    0.0 ± 0.0   
real-book               1.0 ± 0.0    1.0 ± 0.0    0.0 ± 0.0    0.0 ± 0.0   
rock-corpus             1.0 ± 0.0    1.0 ± 0.0    0.0 ± 0.0    0.0 ± 0.0   
schubert-winterreise    1.0 ± 0.0    1.0 ± 0.0    0.0 ± 0.0    0.0 ± 0.0   
when-in-rome            1.0 ± 0.0    1.0 ± 0.0    0.0 ± 0.0    0.0 ± 0.0   
wikifonia             0.92 ± 0.17  0.75 ± 0.35    0.0 ± 0.0    0.1 ± 0.2   

                                   
                            value  
partition                          
biab-internet-corpus    0.0 ± 0.0  
ireal-pro               0.0 ± 0.0  
jazz-corpus             0.0 ± 0.0  
mozart-piano-sonatas   0.15 ± 0.3  
nottingham              0.0 ± 0.0  
real-book               0.0 ± 0.0  
rock-corpus             0.0 ± 0.0  
schubert-winterreise    0.0 ± 0.0  
when-in-rome            0.0 ± 0.0  
wikifonia             0.11 ± 0.18

### Annotation results: audio

In [15]:
_, _, ann_res_audio_str = describe_partitions(all_partitions_anns_audio)
merge_columns(ann_res_audio_str)

In [16]:
ann_res_audio_str

key coverages                       key errorr  \
                              time   duration      value       time   
partition                                                             
billboard                1.0 ± 0.0  1.0 ± 0.0  1.0 ± 0.0  0.0 ± 0.0   
chordify                 1.0 ± 0.0  1.0 ± 0.0  1.0 ± 0.0  0.0 ± 0.0   
isophonics               1.0 ± 0.0  1.0 ± 0.0  1.0 ± 0.0  0.0 ± 0.0   
jaah                     1.0 ± 0.0  1.0 ± 0.0  1.0 ± 0.0  0.0 ± 0.0   
robbie-williams          1.0 ± 0.0  1.0 ± 0.0  1.0 ± 0.0  0.0 ± 0.0   
rwc-pop                          -          -          -          -   
schubert-winterreise     1.0 ± 0.0  1.0 ± 0.0  1.0 ± 0.0  0.0 ± 0.0   
uspop2002                        -          -          -          -   
weimar                   1.0 ± 0.0  1.0 ± 0.0  1.0 ± 0.0  0.0 ± 0.0   

                                           chord coverages               \
                       duration      value            time     duration   
partition                                                                 
billboard             0.0 ± 0.0  0.0 ± 0.0       1.0 ± 0.0    1.0 ± 0.0   
chordify              0.0 ± 0.0  0.0 ± 0.0       1.0 ± 0.0    1.0 ± 0.0   
isophonics            0.0 ± 0.0  0.0 ± 0.0       1.0 ± 0.0    1.0 ± 0.0   
jaah                  0.0 ± 0.0  0.0 ± 0.0      0.95 ± 0.1   0.95 ± 0.1   
robbie-williams       0.0 ± 0.0  0.0 ± 0.0       1.0 ± 0.0    1.0 ± 0.0   
rwc-pop                       -          -       1.0 ± 0.0  0.53 ± 0.45   
schubert-winterreise  0.0 ± 0.0  0.0 ± 0.0       1.0 ± 0.0    1.0 ± 0.0   
uspop2002                     -          -       1.0 ± 0.0   0.3 ± 0.26   
weimar                0.0 ± 0.0  0.0 ± 0.0       1.0 ± 0.0    1.0 ± 0.0   

                                chord errors                          
                          value         time     duration      value  
partition                                                             
billboard             1.0 ± 0.0    0.0 ± 0.0    0.0 ± 0.0  0.0 ± 0.0  
chordify              1.0 ± 0.0    0.0 ± 0.0    0.0 ± 0.0  0.0 ± 0.0  
isophonics            1.0 ± 0.0    0.0 ± 0.0    0.0 ± 0.0  0.0 ± 0.0  
jaah                  1.0 ± 0.0  0.06 ± 0.13  0.06 ± 0.13  0.0 ± 0.0  
robbie-williams       1.0 ± 0.0    0.0 ± 0.0    0.0 ± 0.0  0.0 ± 0.0  
rwc-pop               1.0 ± 0.0    0.0 ± 0.0    0.0 ± 0.0  0.0 ± 0.0  
schubert-winterreise  1.0 ± 0.0    0.0 ± 0.0    0.0 ± 0.0  0.0 ± 0.0  
uspop2002             1.0 ± 0.0    0.0 ± 0.0    0.0 ± 0.0  0.0 ± 0.0  
weimar                1.0 ± 0.0    0.0 ± 0.0    0.0 ± 0.0  0.0 ± 0.0

### LateX tables

In [17]:
print(meta_res_str.to_latex())

\begin{tabular}{lllll}
\toprule
{} & \multicolumn{2}{l}{meta} & \multicolumn{2}{l}{identifiers} \\
{} &    coverage &     accuracy &    coverage &   accuracy \\
partition            &             &              &             &            \\
\midrule
biab-internet-corpus &  0.95 ± 0.1 &  0.92 ± 0.08 &           - &          - \\
billboard            &   1.0 ± 0.0 &    1.0 ± 0.0 &           - &          - \\
chordify             &   1.0 ± 0.0 &    1.0 ± 0.0 &   1.0 ± 0.0 &  1.0 ± 0.0 \\
ireal-pro            &   1.0 ± 0.0 &    1.0 ± 0.0 &           - &          - \\
isophonics           &   1.0 ± 0.0 &    1.0 ± 0.0 &           - &          - \\
jaah                 &  0.8 ± 0.06 &    1.0 ± 0.0 &   1.0 ± 0.0 &  1.0 ± 0.0 \\
jazz-corpus          &   1.0 ± 0.0 &    1.0 ± 0.0 &           - &          - \\
mozart-piano-sonatas &  0.88 ± 0.0 &    1.0 ± 0.0 &   1.0 ± 0.0 &  1.0 ± 0.0 \\
nottingham           &   1.0 ± 0.0 &    1.0 ± 0.0 &           - &          - \\
real-book            &   1.0 ±

/var/folders/q6/phq9pwd56q78175j_yvjz8pc0000gn/T/ipykernel_67980/4075342920.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(meta_res_str.to_latex())


In [18]:
ann_res_score_str.insert(0, "type", "score")
ann_res_audio_str.insert(0, "type", "audio")

In [19]:
ann_res_combined_str = pd.concat([ann_res_audio_str, ann_res_score_str])
ann_res_combined_str

type key coverages                         \
                                     time   duration       value   
partition                                                          
billboard             audio     1.0 ± 0.0  1.0 ± 0.0   1.0 ± 0.0   
chordify              audio     1.0 ± 0.0  1.0 ± 0.0   1.0 ± 0.0   
isophonics            audio     1.0 ± 0.0  1.0 ± 0.0   1.0 ± 0.0   
jaah                  audio     1.0 ± 0.0  1.0 ± 0.0   1.0 ± 0.0   
robbie-williams       audio     1.0 ± 0.0  1.0 ± 0.0   1.0 ± 0.0   
rwc-pop               audio             -          -           -   
schubert-winterreise  audio     1.0 ± 0.0  1.0 ± 0.0   1.0 ± 0.0   
uspop2002             audio             -          -           -   
weimar                audio     1.0 ± 0.0  1.0 ± 0.0   1.0 ± 0.0   
biab-internet-corpus  score     1.0 ± 0.0  1.0 ± 0.0   1.0 ± 0.0   
ireal-pro             score     0.0 ± 0.0  1.0 ± 0.0   1.0 ± 0.0   
jazz-corpus           score     1.0 ± 0.0  1.0 ± 0.0   1.0 ± 0.0   
mozart-piano-sonatas  score    0.5 ± 0.58  0.0 ± 0.0  0.5 ± 0.58   
nottingham            score     1.0 ± 0.0  1.0 ± 0.0   1.0 ± 0.0   
real-book             score     1.0 ± 0.0  1.0 ± 0.0   1.0 ± 0.0   
rock-corpus           score     1.0 ± 0.0  1.0 ± 0.0   1.0 ± 0.0   
schubert-winterreise  score     1.0 ± 0.0  1.0 ± 0.0   1.0 ± 0.0   
when-in-rome          score     1.0 ± 0.0  1.0 ± 0.0   1.0 ± 0.0   
wikifonia             score     1.0 ± 0.0  1.0 ± 0.0   1.0 ± 0.0   

                          key errorr                               \
                                time        duration        value   
partition                                                           
billboard                  0.0 ± 0.0       0.0 ± 0.0    0.0 ± 0.0   
chordify                   0.0 ± 0.0       0.0 ± 0.0    0.0 ± 0.0   
isophonics                 0.0 ± 0.0       0.0 ± 0.0    0.0 ± 0.0   
jaah                       0.0 ± 0.0       0.0 ± 0.0    0.0 ± 0.0   
robbie-williams            0.0 ± 0.0       0.0 ± 0.0    0.0 ± 0.0   
rwc-pop                            -               -            -   
schubert-winterreise       0.0 ± 0.0       0.0 ± 0.0    0.0 ± 0.0   
uspop2002                          -               -            -   
weimar                     0.0 ± 0.0       0.0 ± 0.0    0.0 ± 0.0   
biab-internet-corpus       0.0 ± 0.0       0.0 ± 0.0    0.0 ± 0.0   
ireal-pro                  1.1 ± 0.0       0.0 ± 0.0    0.0 ± 0.0   
jazz-corpus                0.0 ± 0.0       0.0 ± 0.0    0.0 ± 0.0   
mozart-piano-sonatas  62.55 ± 125.03  139.75 ± 83.75  0.25 ± 0.29   
nottingham                 0.0 ± 0.0       0.0 ± 0.0    0.0 ± 0.0   
real-book                  0.0 ± 0.0       0.0 ± 0.0    0.0 ± 0.0   
rock-corpus                0.0 ± 0.0       0.0 ± 0.0    0.0 ± 0.0   
schubert-winterreise       0.0 ± 0.0       0.0 ± 0.0    0.0 ± 0.0   
when-in-rome               0.0 ± 0.0       0.0 ± 0.0    0.0 ± 0.0   
wikifonia                  0.0 ± 0.0       0.0 ± 0.0    0.0 ± 0.0   

                     chord coverages                           chord errors  \
                                time     duration        value         time   
partition                                                                     
billboard                  1.0 ± 0.0    1.0 ± 0.0    1.0 ± 0.0    0.0 ± 0.0   
chordify                   1.0 ± 0.0    1.0 ± 0.0    1.0 ± 0.0    0.0 ± 0.0   
isophonics                 1.0 ± 0.0    1.0 ± 0.0    1.0 ± 0.0    0.0 ± 0.0   
jaah                      0.95 ± 0.1   0.95 ± 0.1    1.0 ± 0.0  0.06 ± 0.13   
robbie-williams            1.0 ± 0.0    1.0 ± 0.0    1.0 ± 0.0    0.0 ± 0.0   
rwc-pop                    1.0 ± 0.0  0.53 ± 0.45    1.0 ± 0.0    0.0 ± 0.0   
schubert-winterreise       1.0 ± 0.0    1.0 ± 0.0    1.0 ± 0.0    0.0 ± 0.0   
uspop2002                  1.0 ± 0.0   0.3 ± 0.26    1.0 ± 0.0    0.0 ± 0.0   
weimar                     1.0 ± 0.0    1.0 ± 0.0    1.0 ± 0.0    0.0 ± 0.0   
biab-internet-corpus      0.95 ± 0.1    1.0 ± 0.0    1.0 ± 

In [20]:
print(ann_res_combined_str.to_latex())

\begin{tabular}{llllllllllllll}
\toprule
{} &   type & \multicolumn{3}{l}{key coverages} & \multicolumn{3}{l}{key errorr} & \multicolumn{3}{l}{chord coverages} & \multicolumn{3}{l}{chord errors} \\
{} &          time &   duration &       value &            time &        duration &        value &            time &     duration &        value &         time &     duration &        value \\
partition            &        &               &            &             &                 &                 &              &                 &              &              &              &              &              \\
\midrule
billboard            &  audio &     1.0 ± 0.0 &  1.0 ± 0.0 &   1.0 ± 0.0 &       0.0 ± 0.0 &       0.0 ± 0.0 &    0.0 ± 0.0 &       1.0 ± 0.0 &    1.0 ± 0.0 &    1.0 ± 0.0 &    0.0 ± 0.0 &    0.0 ± 0.0 &    0.0 ± 0.0 \\
chordify             &  audio &     1.0 ± 0.0 &  1.0 ± 0.0 &   1.0 ± 0.0 &       0.0 ± 0.0 &       0.0 ± 0.0 &    0.0 ± 0.0 &       1.0 ± 0.0 &    1.0 ± 0.0 &  

/var/folders/q6/phq9pwd56q78175j_yvjz8pc0000gn/T/ipykernel_67980/1135342056.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(ann_res_combined_str.to_latex())


In [21]:
print(ann_res_score_str.to_latex())

\begin{tabular}{llllllllllllll}
\toprule
{} &   type & \multicolumn{3}{l}{key coverages} & \multicolumn{3}{l}{key errorr} & \multicolumn{3}{l}{chord coverages} & \multicolumn{3}{l}{chord errors} \\
{} &          time &   duration &       value &            time &        duration &        value &            time &     duration &        value &         time &     duration &        value \\
partition            &        &               &            &             &                 &                 &              &                 &              &              &              &              &              \\
\midrule
biab-internet-corpus &  score &     1.0 ± 0.0 &  1.0 ± 0.0 &   1.0 ± 0.0 &       0.0 ± 0.0 &       0.0 ± 0.0 &    0.0 ± 0.0 &      0.95 ± 0.1 &    1.0 ± 0.0 &    1.0 ± 0.0 &   0.05 ± 0.1 &    0.0 ± 0.0 &    0.0 ± 0.0 \\
ireal-pro            &  score &     0.0 ± 0.0 &  1.0 ± 0.0 &   1.0 ± 0.0 &       1.1 ± 0.0 &       0.0 ± 0.0 &    0.0 ± 0.0 &       0.2 ± 0.4 &    1.0 ± 0.0 &  

/var/folders/q6/phq9pwd56q78175j_yvjz8pc0000gn/T/ipykernel_67980/1932936821.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(ann_res_score_str.to_latex())


In [22]:
print(ann_res_audio_str.to_latex())

\begin{tabular}{llllllllllllll}
\toprule
{} &   type & \multicolumn{3}{l}{key coverages} & \multicolumn{3}{l}{key errorr} & \multicolumn{3}{l}{chord coverages} & \multicolumn{3}{l}{chord errors} \\
{} &          time &   duration &      value &       time &   duration &      value &            time &     duration &      value &         time &     duration &      value \\
partition            &        &               &            &            &            &            &            &                 &              &            &              &              &            \\
\midrule
billboard            &  audio &     1.0 ± 0.0 &  1.0 ± 0.0 &  1.0 ± 0.0 &  0.0 ± 0.0 &  0.0 ± 0.0 &  0.0 ± 0.0 &       1.0 ± 0.0 &    1.0 ± 0.0 &  1.0 ± 0.0 &    0.0 ± 0.0 &    0.0 ± 0.0 &  0.0 ± 0.0 \\
chordify             &  audio &     1.0 ± 0.0 &  1.0 ± 0.0 &  1.0 ± 0.0 &  0.0 ± 0.0 &  0.0 ± 0.0 &  0.0 ± 0.0 &       1.0 ± 0.0 &    1.0 ± 0.0 &  1.0 ± 0.0 &    0.0 ± 0.0 &    0.0 ± 0.0 &  0.0 ± 0.0 \\
isophoni

/var/folders/q6/phq9pwd56q78175j_yvjz8pc0000gn/T/ipykernel_67980/937250796.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(ann_res_audio_str.to_latex())


In [24]:
print(ann_res_score_str.to_latex)

<bound method NDFrame.to_latex of                        type key coverages                         \
                                     time   duration       value   
partition                                                          
biab-internet-corpus  score     1.0 ± 0.0  1.0 ± 0.0   1.0 ± 0.0   
ireal-pro             score     0.0 ± 0.0  1.0 ± 0.0   1.0 ± 0.0   
jazz-corpus           score     1.0 ± 0.0  1.0 ± 0.0   1.0 ± 0.0   
mozart-piano-sonatas  score    0.5 ± 0.58  0.0 ± 0.0  0.5 ± 0.58   
nottingham            score     1.0 ± 0.0  1.0 ± 0.0   1.0 ± 0.0   
real-book             score     1.0 ± 0.0  1.0 ± 0.0   1.0 ± 0.0   
rock-corpus           score     1.0 ± 0.0  1.0 ± 0.0   1.0 ± 0.0   
schubert-winterreise  score     1.0 ± 0.0  1.0 ± 0.0   1.0 ± 0.0   
when-in-rome          score     1.0 ± 0.0  1.0 ± 0.0   1.0 ± 0.0   
wikifonia             score     1.0 ± 0.0  1.0 ± 0.0   1.0 ± 0.0   

                          key errorr                               \
            